In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [ ]:
import rmsd as rmsdlib

In [ ]:
rmsdlib.__version__

# Directories

In [ ]:
resources = Path("../tests/resources/").resolve()

# Plot

In [ ]:
import plot_funcs

# Read file

In [ ]:
# https://pubchem.ncbi.nlm.nih.gov/compound/15634633#section=Other-Identifiers
filename_a = resources / "examples" / "NS00006345.xyz"
filename_a = resources / "examples" / "CHEMBL135626.xyz"

In [ ]:
atoms_a, coord_a = rmsdlib.get_coordinates_xyz(filename_a, return_atoms_as_int=True)
coord_a -= rmsdlib.centroid(coord_a)

# Ignore Hydrogens
view_hydrogens, = np.where(atoms_a != 1)

atoms_a = atoms_a[view_hydrogens]
coord_a = coord_a[view_hydrogens]

# Just change the molecule a little bit. Alchemically change Carbon to Nitrogen
#atoms_a[1] = 7 

In [ ]:
fig_width = np.max([np.max(coord_a), -np.min(coord_a)]) * 1.2
fig_width = 2.0

In [ ]:
fig, ax = plot_funcs.get_plot()

print(fig_width)

plot_funcs.plot_coord(ax, atoms_a, coord_a)

ax.set_xlim(-fig_width, fig_width)
ax.set_ylim(-fig_width, fig_width)
pass

In [ ]:
def box68(atoms_a, coord_a):
    
    fig, axs = plot_funcs.get_plot(4)
    ax1, ax2, ax3, ax4 = axs

    ax1.set_xlim(-fig_width, fig_width)
    ax1.set_ylim(-fig_width, fig_width)
    ax1.grid(color='grey', linestyle='-', linewidth=0.5)
    ax2.grid(color='grey', linestyle='-', linewidth=0.5)
    ax3.grid(color='grey', linestyle='-', linewidth=0.5)
    ax4.grid(color='grey', linestyle='-', linewidth=0.5)
    
    # Coord A
    centroid_a = rmsdlib.centroid(coord_a)
    coord_a -= centroid_a
    inertia_a = rmsdlib.get_inertia_tensor(atoms_a, coord_a)
    centroid_a = rmsdlib.centroid(coord_a)
    
    eigval_a, eigvec_a = np.linalg.eig(inertia_a)
    eigvec_a = eigvec_a[np.argsort(eigval_a)]
    
    print("A:")
    print("Coord", coord_a)
    print("in ", inertia_a)
    print("val", eigval_a)
    print("vec" , eigvec_a)
    
    plot_funcs.plot_coord(ax1, atoms_a, np.dot(coord_a, eigvec_a), show_hydrogens=False)
    plot_funcs.do_dot(ax1, centroid[0], centroid[1], size=4)
    plot_funcs.do_arrow(ax1, centroid_a, eigvec_a[0][:2], color="red")
    plot_funcs.do_arrow(ax1, centroid_a, eigvec_a[1][:2], color="blue")


    
    # Create coord B
    U = coord_funcs.rotation_matrix(100)
    _xy = np.dot(coord_a[:,:2], U)
    coord_b = coord_a.copy() # copy?
    coord_b[:,:2] = _xy
    
    coord_b -= rmsdlib.centroid(coord_b)
    centroid_b = rmsdlib.centroid(coord_b)
    
    inertia_b = rmsdlib.get_inertia_tensor(atoms_a, coord_b)
    eigval_b, eigvec_b = np.linalg.eig(inertia_b)
    #eigvec_b = eigvec_b[np.argsort(eigval_b)]
    
    print("B:")
    print("coord", coord_b)
    print("in ", inertia_b)
    print("val", eigval_b)
    print("vec", eigvec_b)
    
    # TODO Calculate angle difference between eigval a and b, assert == choosen angle
    # TODO Rotate eigval
    
    plot_funcs.plot_coord(ax2, atoms_a, coord_b, show_hydrogens=False)
    plot_funcs.do_dot(ax2, centroid_b[0], centroid_b[1], size=4)
    plot_funcs.do_arrow(ax2, centroid_b, eigvec_b[0][:2], color="red")
    plot_funcs.do_arrow(ax2, centroid_b, eigvec_b[1][:2], color="blue")


    print("C:")


    print("det:", np.linalg.det(eigvec_b))

    if np.linalg.det(eigvec_b) < 0:
        eigvec_b[1,:] = -eigvec_b[1,:]
    
    coord_c = np.dot(coord_b, eigvec_b)
    inertia_c = rmsdlib.get_inertia_tensor(atoms_a, coord_c)
    eigval_c, eigvec_c = np.linalg.eig(inertia_c)
    #rotation_matrix = eigenvectors.T

    
    print(coord_b)

    plot_funcs.plot_coord(ax3, atoms_a, coord_c, show_hydrogens=False)
    plot_funcs.do_dot(ax3, centroid_b[0], centroid_b[1], size=4)
    plot_funcs.do_arrow(ax3, centroid_c, eigvec_c[0][:2], color="red")
    plot_funcs.do_arrow(ax3, centroid_c, eigvec_c[1][:2], color="blue")
    

    print("D:")

    U = coord_funcs.rotation_matrix(20)
    _xy = np.dot(coord_a[:,:2], U)
    coord_d = coord_a.copy() # copy?
    coord_d[:,:2] = _xy
    
    coord_d -= rmsdlib.centroid(coord_b)
    centroid_d = rmsdlib.centroid(coord_b)
    
    inertia_d = rmsdlib.get_inertia_tensor(atoms_a, coord_d)
    eigval_d, eigvec_d = np.linalg.eig(inertia_d)
    #eigvec_d = eigvec_d[np.argsort(eigval_d)[::-1]]

    print("det:", np.linalg.det(eigvec_d))

    # Step 3: Construct the rotation matrix from eigenvectors
    rotation_matrix = eigvec_d.T  # Use the transpose of eigenvectors

    if np.linalg.det(rotation_matrix) < 0:
        rotation_matrix[:, 1] = -rotation_matrix[:, 1]  # Flip the second eigenvector if determinant is negative

    coord_d = np.dot(rotation_matrix, coord_d.T).T
    
    #coord_d = np.dot(coord_d, eigvec_d) # Rotate with inertia eigvec
    inertia_d = rmsdlib.get_inertia_tensor(atoms_a, coord_d)
    eigval_d, eigvec_d = np.linalg.eig(inertia_d)
    print(eigvec_d)

    plot_funcs.plot_coord(ax4, atoms_a, coord_d)
    plot_funcs.do_dot(ax4, centroid_d[0], centroid_d[1], size=4)
    plot_funcs.do_arrow(ax4, centroid_d, eigvec_d[0][:2], color="red")
    plot_funcs.do_arrow(ax4, centroid_d, eigvec_d[1][:2], color="blue")
    
    
box68(atoms_a, coord_a)

In [ ]:
# Plot rotation
fig, ax = plot_funcs.get_plot()

centroid = np.asarray([0, 0])
radius = 1.0

p1 = np.array(centroid)
p1[0] += radius
p1[1] += radius

degrees = 80
step_size = 5
n_steps = int(degrees / step_size)

p2 = p1
for d in range(n_steps):
    U = coord_funcs.rotation_matrix(step_size)
    p3 = np.dot(p2, U)
    plot_funcs.do_arrow(ax, p2, p3, rad=.01)
    p2 = p3

plot_funcs.do_dot(ax, *centroid)
plot_funcs.do_dot(ax, *p1)
plot_funcs.do_dot(ax, *p2)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)

pass

In [ ]:
fig, axs = plot_funcs.get_plot(1)
ax1 = axs

# Coord A
centroid_a = rmsdlib.centroid(coord_a)
coord_a -= centroid_a
inertia_a = rmsdlib.get_inertia_tensor(atoms_a, coord_a)
centroid_a = rmsdlib.centroid(coord_a)
eigval_a, eigvec_a = np.linalg.eig(inertia_a)
eigvec_a = eigvec_a[np.argsort(eigval_a)]


coord_b = np.array(coord_a)

# Create coord B
total = 5
cmap1 = cm.autumn
cmap2 = cm.cool
norm = Normalize(vmin=1, vmax=total*2)
for i in range(1, total+1):

    U = coord_funcs.rotation_matrix(20)
    _xy = np.dot(coord_b[:,:2], U)
    coord_b = np.array(coord_b)
    coord_b[:,:2] = _xy
    coord_b += 5
    
    centroid_b = rmsdlib.centroid(coord_b)
    coord_b -= centroid_b
    centroid_b = rmsdlib.centroid(coord_b)
    inertia_b = rmsdlib.get_inertia_tensor(atoms_a, coord_b)
    eigval_b, eigvec_b = np.linalg.eig(inertia_b)
    eigvec_b = eigvec_b[np.argsort(eigval_b)]

    #plot_funcs.plot_coord(ax2, atoms_a, coord_b, show_hydrogens=False)
    #plot_funcs.do_dot(ax2, centroid_b[0], centroid_b[1], size=4)
    plot_funcs.do_arrow(ax1, centroid_b, eigvec_b[0], rad=.01, color=cmap1(norm(i)))
    plot_funcs.do_arrow(ax1, centroid_b, eigvec_b[1], rad=.01, color=cmap2(norm(i)))

    #plot_funcs.do_dot(ax2, centroid_b[0], centroid_b[1], size=4)


ax1.set_xlim(-fig_width, fig_width)
ax1.set_ylim(-fig_width, fig_width)

pass